### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
            
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10) # use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 41.164386
2 		 13.379747
3 		 3.910205
4 		 2.770485
5 		 2.627602
6 		 2.572976
7 		 2.528115
8 		 2.480498
9 		 2.427335
10 		 2.367746


In [4]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.15917645  8.96325653 10.41467262 10.19862256]
 [ 8.48878384  7.07152481  9.4277672   8.80347493]
 [ 7.85580596  7.06368553  8.12097671  7.83632846]
 [ 9.51645585  7.97788847 10.26325165  9.5718359 ]
 [ 8.52698144  7.27112169  9.11142489  8.52496346]
 [ 6.68580361  5.64499443  7.40211536  7.00649089]
 [ 9.09516583  7.97727608  9.88711629  9.12155517]
 [ 8.37289792  6.70115532  9.01385387  8.02743427]
 [ 8.53566651  7.07001707  9.25550415  8.34560015]
 [ 7.54007513  6.44098742  7.9923876   7.99932245]
 [ 8.97773813  8.04186186  9.21116686  8.78287809]
 [ 9.54412441  8.1236192  10.12047215  9.70191699]
 [ 9.38218767  8.56771143  9.64393541  9.03191385]
 [ 6.76939362  6.16283935  6.64305682  6.58975062]
 [ 9.85705488  8.65773101 10.0515888   9.7609467 ]
 [ 9.72932793  8.58757626 10.00382441  9.91355199]
 [ 8.61962522  7.64534618  8.36865     8.160365  ]
 [ 7.08163569  6.89862646  6.61294191  7.38152314]
 [ 8.11196735  7.4571769   8.71271894  8.53355259]
 [ 8.15385762  6.69396951  8.83

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 45.940333
2 		 16.465400
3 		 4.251444
4 		 2.703147
5 		 2.539808
6 		 2.473496
7 		 2.417586
8 		 2.359233
9 		 2.295238
10 		 2.224810
11 		 2.147973
12 		 2.065214
13 		 1.977402
14 		 1.885748
15 		 1.791722
16 		 1.696932
17 		 1.602976
18 		 1.511276
19 		 1.422944
20 		 1.338694
21 		 1.258821
22 		 1.183241
23 		 1.111584
24 		 1.043307
25 		 0.977809
26 		 0.914525
27 		 0.852994
28 		 0.792899
29 		 0.734079
30 		 0.676519
31 		 0.620332
32 		 0.565729
33 		 0.512984
34 		 0.462406
35 		 0.414303
36 		 0.368959
37 		 0.326611
38 		 0.287435
39 		 0.251538
40 		 0.218952
41 		 0.189642
42 		 0.163508
43 		 0.140400
44 		 0.120124
45 		 0.102461
46 		 0.087173
47 		 0.074016
48 		 0.062752
49 		 0.053149
50 		 0.044992
51 		 0.038083
52 		 0.032245
53 		 0.027320
54 		 0.023169
55 		 0.019673
56 		 0.016727
57 		 0.014244
58 		 0.012151
59 		 0.010383
60 		 0.008888
61 		 0.007622
62 		 0.006548
63 		 0.005636
64 		

In [6]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.99999946  9.99999936  9.99999976 10.00000088]
 [ 9.99999917  3.99999923  8.99999971 10.00000119]
 [ 7.99999951  8.99999949  9.99999982  5.00000076]
 [ 9.00000046  8.00000047 10.0000002   9.99999934]
 [ 9.99999981  4.9999997   8.99999991  9.0000004 ]
 [ 6.00000063  4.00000059 10.00000024  5.99999913]
 [ 8.99999958  7.99999965  9.99999988  9.00000058]
 [10.00000017  5.0000003   9.00000012  7.99999967]
 [ 6.99999948  7.99999958  9.99999985  8.0000007 ]
 [ 9.00000029  5.00000031  9.00000013  6.99999958]
 [ 9.00000001  7.99999995 10.          8.00000006]
 [ 8.99999917  9.99999925  9.99999973  9.00000118]
 [10.0000014   9.00000148 10.00000058  7.9999979 ]
 [ 5.00000053  8.00000054  5.00000021  7.99999922]
 [ 9.99999986  7.99999976  9.99999991 10.00000029]
 [ 9.00000068  9.00000076 10.00000027  9.99999893]
 [ 8.99999837  7.9999983   7.99999936  8.00000246]
 [10.00000036  8.00000038  1.00000016  9.99999948]
 [ 4.99999989  5.99999988  9.99999996 10.00000018]
 [ 8.00000068  7.00000065 10.00

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [7]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [8]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 41.014873
2 		 13.864336
3 		 4.048648
4 		 2.886156
5 		 2.772759
6 		 2.736065
7 		 2.709042
8 		 2.682781
9 		 2.654652
10 		 2.623408
11 		 2.588207
12 		 2.548371
13 		 2.503334
14 		 2.452651
15 		 2.396046
16 		 2.333461
17 		 2.265112
18 		 2.191539
19 		 2.113623
20 		 2.032574
21 		 1.949866
22 		 1.867129
23 		 1.786001
24 		 1.707964
25 		 1.634207
26 		 1.565521
27 		 1.502266
28 		 1.444400
29 		 1.391553
30 		 1.343137
31 		 1.298460
32 		 1.256814
33 		 1.217554
34 		 1.180141
35 		 1.144160
36 		 1.109328
37 		 1.075477
38 		 1.042545
39 		 1.010548
40 		 0.979563
41 		 0.949703
42 		 0.921097
43 		 0.893873
44 		 0.868146
45 		 0.844005
46 		 0.821507
47 		 0.800674
48 		 0.781494
49 		 0.763926
50 		 0.747898
51 		 0.733322
52 		 0.720093
53 		 0.708100
54 		 0.697225
55 		 0.687355
56 		 0.678381
57 		 0.670197
58 		 0.662710
59 		 0.655832
60 		 0.649485
61 		 0.643602
62 		 0.638119
63 		 0.632985
64 		

In [9]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 8.50739851927269:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [10]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20) #fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.907640
2 		 10.929235
3 		 7.446629
4 		 5.727707
5 		 4.650101
6 		 3.896183
7 		 3.334738
8 		 2.899732
9 		 2.553333
10 		 2.271958
11 		 2.040024
12 		 1.846728
13 		 1.684260
14 		 1.546771
15 		 1.429743
16 		 1.329603
17 		 1.243472
18 		 1.169000
19 		 1.104257
20 		 1.047652


`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [11]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users)) # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [12]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
